# Установка и настройка Spark

Структура шага

* скринкаст cluster_spark (прервать на месте "сервис пошел стартовать, это займет какое-то время - 01:32)
    * по нему слайд "настройка кластера"
    * чуть деталей в материалах
* скринкаст spark_setup 
    * по нему слайд "запуск spark приложений"
    * чуть деталей в материалах
* скринкаст spark_jupyter
    * по нему слайд "настройки jupyter notebook"
    * детали в материалах
    * слайд "web интерфейс"
    * детали
* материалы
    * настрока спарка локально
    * JDBC
    * управление параллелизмом

### Настройка кластера

* добавляем сервис Spark
* (можно остановить Hue, если мало памяти)
* spark готов к работе

### Как увеличить объем используемой памяти

После запуска одноименного сервиса, Spark готов к работе. Поскольку на нашем кластере все процессы работают на одном узле, этому узлу может не хватить оперативной памяти. Увеличим объем с 2ГБ до 3ГБ, этого достаточно для того, чтобы работали все наши примеры.

* в кластер менеджере заходим в сервис Yarn
* вкладка Configuration
* находим параметр "container memory"

![](hadoop_yarn_memory.PNG)

* устанавливаем новое значение 3
* сохраняем изменения
* (через минуту) появляется иконка "перезагрузить" 

![](yarn_restart.PNG)

* кликаем ее, дожидаемся перезагрузки Yarn
* в WEB интерфейсе менеджера ресурсов должно появиться значение "3GB" в верхней строке ("Memory Total")

### Запуск Spark приложений

* запуск интерактивного shell (scala или python)
    * spark-shell (scala)
    * pyspark (python)
* запуск скомпилированных приложений (JAR)
    * spark-submit
* мы будем использовать pyspark, а точнее - jupyter notebook

### Параметры запуска Spark приложений

Для того, чтобы на нашем маленьком кластере работали Spark приложения, необходимо задать следующие параметры

* --driver-memory 1G: объем памяти, выделяемый драйверу на узле кластера
* --executor-memory 600M: объем памяти, выделяемый экзекьютору

Эти параметры нужно задать

* в командной строке (при запуске интерактивного spark shell-а)

`pyspark --driver-memory 1G --executor-memory 600M`

* в качестве значения переменной окружения `PYSPARK_SUBMIT_ARGS` (при работе со spark из jupyter notebook)

`os.environ["PYSPARK_SUBMIT_ARGS"] = --driver-memory 1G --executor-memory 600M pyspark-shell`


### Настройки jupyter notebook

* стандартная "шапка" 
* создание сессии
* интерактивная работа (как обычно в jupyter)
* остановка сессии
* перезапуск ядра (если что-то нужно поменять в конфигурации)

К материалам курса приложен jupyter notebook с примером, продемонстрированным в скринкасте.

### Web интерфейс

* доступен на порту 8088 (в кластер менеджере) или на порту 4040 (локальная установка)
* в кластере - через Resource Manager-а (с возможностью принудительной остановки)
* можно увидеть процесс выполнения job-а
* использование кэша (разберем чуть позже)
* информацию о процессах (драйвере и экзекьюторах)

### Локальная установка Spark

Spark может работать локально (т.е. может быть установлен на рабочем месте, в нашем случае - не в виртуальной машине. Это часто используется для тестирования). В модуле мы будем использовать как кластерный вариант Spark, так и локальный.

Как установить и настроить spark для локальной работы:

1. Должна быть предустановлена java (установить, если это не так)
2. Установить spark (см. инструкции на сайте проекта), он уже может быть установлен (с другими продуктами -  у меня был установлен вместе со scala)
3. pip3 install pyspark
4. pip3 install ipyparallel

Далее в ноутбуке

Стандартная шапка:

    import os
    from pyspark.sql import SparkSession
    
    os.environ["SPARK_HOME"] = "<путь куда был установлен pyspark>"
    os.environ["PYSPARK_PYTHON"] = "/usr/bin/python3"
    os.environ["PYSPARK_DRIVER_PYTHON"] = "python3"
    os.environ["PYSPARK_SUBMIT_ARGS"] = "pyspark-shell"

Чтобы стартануть сессию делаем

    sp = SparkSession.builder.master("local").appName("имя_приложения").getOrCreate()

После этого сессию можно видеть здесь:

    http://localhost:4040

Чтобы остановить сессию делаем

    sp.stop()

После этого сессия пропадает...

### Настройка Spark для работы с JDBC источниками

Для работы с реляционными базами данных (например, для чтения данных из них) необходимо установить JDBC драйвер и указать Spark два параметра

* --driver-class-path <путь к JDBC драйверу>
* --jars <путь к JDBC драйверу>

(один и тот же пусть указываем два раза). **ВАЖНО** файл с драйвером должен находиться в одном и том же месте (директории) на всех узлах кластера, на которых будут работать executor-ы, его использующие.

Например, для работы с PostgreSQL в нашем кластере нужно будет указать такую комбинацию параметров (см. способы задания параметров выше):

`--driver-class-path /usr/share/java/postgresql.jar --jars /usr/share/java/postgresql.jar

см. более подробно https://spark.apache.org/docs/latest/sql-data-sources-jdbc.html

Мы разберем примеры работы с базами данных в Spark более подробно на следующем шаге этого модуля.


### Основные параметры параллелизма Spark

Spark подразумевает параллельную обработку, как мы уже говорили ранее, dataframe состоит из разделов (`partitions`), каждый из которых хранится и обрабатывается на своем узле кластера. Как правило, количество разделов можно задать в момент создания dataframe. В работе инженера данных наиболее частым способом создания dataframe является его загрузка (из источника - файла или таблицы). Задать количество разделов можно с помощью опции `numPartitions` (более подробно с опциями мы познакомимся на следующем шаге).

С другой стороны для параллельной обработки нужны "обработчики" (`executors` в терминологии spark). Их количество задается в параметре `--num-executors` при запуске spark приложения. 

Другими параметрами, связанными с параллельной обработкой, являются

* --executor-cores: количество ядер, выделяемых одному executor-у
* --executor-memory: объем оперативной памяти, выделяемый одному executor-у

**ВАЖНО** некорретное задание этих параметров может привести к одному из следующих исходов (решает логика Spark)

* приложение не будет запущено 
* приложение будет запущено с другими значениями параметров параллелизма (их установит spark)

Хорошая статья про partitions: https://medium.com/parrot-prediction/partitioning-in-apache-spark-8134ad840b0